In [1]:
import pandas as pd

# Load soil data
soil = pd.read_csv("C:/Users/lenovo/Downloads/water resource managment/soil_data.csv")
print("Soil Data:")
print(soil.head())

# Load train timeseries data
train = pd.read_csv("C:/Users/lenovo/Downloads/water resource managment/train_timeseries.csv")
print("Train Timeseries Data:")
print(train.head())



Soil Data:
   fips        lat        lon  elevation  slope1  slope2  slope3  slope4  \
0  1001  32.536382 -86.644490         63  0.0419  0.2788  0.2984  0.2497   
1  1005  31.870670 -85.405456        146  0.0158  0.1868  0.5441  0.2424   
2  1003  30.659218 -87.746067         52  0.0746  0.4370  0.4415  0.0469   
3  1007  33.015893 -87.127148         93  0.0144  0.1617  0.3714  0.3493   
4  1009  33.977448 -86.567246        198  0.0050  0.0872  0.2799  0.3576   

   slope5  slope6  ...  CULTRF_LAND  CULTIR_LAND  CULT_LAND  SQ1  SQ2  SQ3  \
0  0.1142  0.0170  ...    56.293411     1.014811  57.308224    1    1    1   
1  0.0106  0.0003  ...    72.578804     1.828159  74.406960    3    2    1   
2  0.0000  0.0000  ...    59.843639     2.996914  62.840553    3    2    1   
3  0.0898  0.0134  ...     1.916593     0.008330   1.924924    3    2    1   
4  0.1477  0.1037  ...     1.891909     0.027488   1.919397    3    2    1   

   SQ4  SQ5  SQ6  SQ7  
0    1    1    1    2  
1    1    1    

In [2]:
print(train.isnull().sum())


fips                  0
date                  0
PRECTOT               0
PS                    0
QV2M                  0
T2M                   0
T2MDEW                0
T2MWET                0
T2M_MAX               0
T2M_MIN               0
T2M_RANGE             0
TS                    0
WS10M                 0
WS10M_MAX             0
WS10M_MIN             0
WS10M_RANGE           0
WS50M                 0
WS50M_MAX             0
WS50M_MIN             0
WS50M_RANGE           0
score          16543884
dtype: int64


In [3]:
train_labeled = train.dropna(subset=["score"])
print(train_labeled.shape)


(2756796, 21)


In [4]:
print(train_labeled["score"].value_counts())


score
0.0000    1480827
1.0000     219135
2.0000     123789
3.0000      82801
4.0000      45841
           ...   
0.1145          1
2.0172          1
0.6750          1
1.3998          1
0.6060          1
Name: count, Length: 55395, dtype: int64


In [5]:
merged = train_labeled.merge(soil, on="fips", how="left")
print(merged.shape)



(2756796, 52)


In [6]:

merged["score_class"] = merged["score"].round().clip(0, 4).astype(int)
print(merged["score_class"].value_counts())


score_class
0    1652230
1     466944
2     295331
3     196802
4     145489
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

X = merged.drop(columns=["score", "score_class", "date", "fips"])  # features
y = merged["score_class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(
    n_estimators=100,
    class_weight="balanced",  # handles imbalance
    random_state=42,
    n_jobs=-1
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.69      0.97      0.81    330447
           1       0.62      0.22      0.32     93389
           2       0.59      0.22      0.32     59066
           3       0.60      0.24      0.34     39360
           4       0.68      0.36      0.48     29098

    accuracy                           0.68    551360
   macro avg       0.64      0.40      0.45    551360
weighted avg       0.66      0.68      0.62    551360



In [9]:
# Convert date to datetime first
merged["date"] = pd.to_datetime(merged["date"])

# Extract useful numeric features from date
merged["year"] = merged["date"].dt.year
merged["month"] = merged["date"].dt.month
merged["dayofyear"] = merged["date"].dt.dayofyear

# Drop the original date column so only numeric features remain
X = merged.drop(["score_class", "date"], axis=1)
y = merged["score_class"]


In [14]:
# Drop columns that cause leakage
X = merged.drop(["score_class", "date", "score"], axis=1)
y = merged["score_class"]


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 1. Convert date to datetime
merged["date"] = pd.to_datetime(merged["date"])

# 2. Extract useful features
merged["year"] = merged["date"].dt.year
merged["month"] = merged["date"].dt.month
merged["dayofyear"] = merged["date"].dt.dayofyear

# 3. Define X and y (drop "date" to avoid dtype error!)
X = merged.drop(["score_class", "date"], axis=1)
y = merged["score_class"]

# 4. Check if ANY non-numeric columns are left
print(X.dtypes)

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


fips               int64
PRECTOT          float64
PS               float64
QV2M             float64
T2M              float64
T2MDEW           float64
T2MWET           float64
T2M_MAX          float64
T2M_MIN          float64
T2M_RANGE        float64
TS               float64
WS10M            float64
WS10M_MAX        float64
WS10M_MIN        float64
WS10M_RANGE      float64
WS50M            float64
WS50M_MAX        float64
WS50M_MIN        float64
WS50M_RANGE      float64
score            float64
lat              float64
lon              float64
elevation          int64
slope1           float64
slope2           float64
slope3           float64
slope4           float64
slope5           float64
slope6           float64
slope7           float64
slope8           float64
aspectN          float64
aspectE          float64
aspectS          float64
aspectW          float64
aspectUnknown    float64
WAT_LAND         float64
NVG_LAND         float64
URB_LAND         float64
GRS_LAND         float64


In [23]:
print(X.dtypes)


fips               int64
PRECTOT          float64
PS               float64
QV2M             float64
T2M              float64
T2MDEW           float64
T2MWET           float64
T2M_MAX          float64
T2M_MIN          float64
T2M_RANGE        float64
TS               float64
WS10M            float64
WS10M_MAX        float64
WS10M_MIN        float64
WS10M_RANGE      float64
WS50M            float64
WS50M_MAX        float64
WS50M_MIN        float64
WS50M_RANGE      float64
score            float64
lat              float64
lon              float64
elevation          int64
slope1           float64
slope2           float64
slope3           float64
slope4           float64
slope5           float64
slope6           float64
slope7           float64
slope8           float64
aspectN          float64
aspectE          float64
aspectS          float64
aspectW          float64
aspectUnknown    float64
WAT_LAND         float64
NVG_LAND         float64
URB_LAND         float64
GRS_LAND         float64


In [24]:
X.select_dtypes(exclude=["number"]).head()


""
0
1
2
3
4


In [25]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    330447
           1       1.00      1.00      1.00     93389
           2       1.00      1.00      1.00     59066
           3       1.00      1.00      1.00     39360
           4       1.00      1.00      1.00     29098

    accuracy                           1.00    551360
   macro avg       1.00      1.00      1.00    551360
weighted avg       1.00      1.00      1.00    551360

[[330447      0      0      0      0]
 [     0  93388      1      0      0]
 [     0      2  59056      8      0]
 [     0      0     17  39331     12]
 [     0      0      1    115  28982]]


In [26]:
import pandas as pd

importances = pd.Series(model.feature_importances_, index=X.columns)
print(importances.sort_values(ascending=False).head(10))


score        0.741501
year         0.029625
lon          0.014698
lat          0.010917
dayofyear    0.010698
T2M_MAX      0.010246
T2M_RANGE    0.010193
PS           0.009584
TS           0.008691
T2M          0.007867
dtype: float64
